In [4]:
import datasets
import random

dataset_name = "natural_questions"
config_name = "default"
revision = "main"
split = None
max_train_samples = None
max_eval_samples = None
cache_dir = None

# raw_datasets = datasets.load_dataset(
#     dataset_name, config_name,
#     revision=revision,
#     split=split,
#     cache_dir=cache_dir,
# )

raw_datasets = datasets.load_dataset(
    "json", 
    data_files={"train": "/dccstor/srosent3/reranking/sap_genq/nq_format_new/train/synthetic_questions_baseformat_*", "validation": "/dccstor/srosent3/reranking/sap_genq/nq_format_new/dev/synthetic_questions_baseformat_*"}, 
    split=split,
    cache_dir=cache_dir
)



train_examples = raw_datasets["train"]
max_train_samples = max_train_samples
if max_train_samples is not None:
    # We will select sample from whole data if argument is specified
    train_examples = train_examples.select(range(max_train_samples))

print(f"Using {train_examples.num_rows} train examples.")

eval_examples = raw_datasets["validation"]
max_eval_samples = max_eval_samples
if max_eval_samples is not None:
    # We will select sample from whole data if argument is specified
    random_idxs = random.sample(range(len(eval_examples)), max_eval_samples)
    eval_examples = eval_examples.select(random_idxs)

print(f"Using {eval_examples.num_rows} eval examples.")
print("DONE")
average_length(train_examples)

Using custom data configuration default-35fbc6b7d6199ea2
Reusing dataset json (/dccstor/srosent3/huggingface_cache/datasets/json/default-35fbc6b7d6199ea2/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/2 [00:00<?, ?it/s]

Using 47298 train examples.
Using 2000 eval examples.
DONE


AttributeError: 'list' object has no attribute 'split'

In [ ]:
print("hello")

compute length of answer span and passage answer

In [ ]:
def countX(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count

def average_length(examples):
    short_answer_html_len  = 0
    short_answer_len = 0
    long_answer_html_len  = 0
    long_answer_len = 0
    short_answer_count = 0
    long_answer_count = 0
    index = 0
    for example in examples:
        index += 1
        if 'short_answers' in example['annotations']:
            for short_answer in example['annotations']['short_answers']:
                if len(short_answer['start_token']) == 0:
                    continue
                short_answer_count += 1
                tokens = example['document']['tokens']['token'][short_answer['start_token'][0]:short_answer['end_token'][0]]
                is_html = example['document']['tokens']['is_html'][short_answer['start_token'][0]:short_answer['end_token'][0]]
                short_answer_html_len += len(tokens)
                short_answer_len += countX(is_html, False)
        if 'long_answer' in example['annotations']:
            for long_answer in example['annotations']['long_answer']:
                if long_answer['candidate_index'] == -1:
                    continue
                long_answer_count += 1
                tokens = example['document']['tokens']['token'][long_answer['start_token']:long_answer['end_token']]
                is_html = example['document']['tokens']['is_html'][long_answer['start_token']:long_answer['end_token']]
                long_answer_html_len += len(tokens)
                long_answer_len += countX(is_html, False)
        if index % 5000 == 0:
            print(f'short: {short_answer_count} long: {long_answer_count} all: {index}')
            print(f'short answer w/html: {short_answer_html_len/short_answer_count}')
            print(f'short answer: {short_answer_len/short_answer_count}')
            print(f'long answer w/html: {long_answer_html_len/long_answer_count}')
            print(f'long answer: {long_answer_len/long_answer_count}')

average_length(train_examples)

short: 1708 long: 2455 all: 5000
short answer w/html: 4.1703747072599535
short answer: 4.164519906323185
long answer w/html: 358.24236252545825
long answer: 251.6826883910387
short: 3493 long: 4956 all: 10000
short answer w/html: 4.192384769539078
short answer: 4.188376753507014
long answer w/html: 362.6412429378531
long answer: 254.0536723163842
short: 5273 long: 7451 all: 15000
short answer w/html: 4.197231177697705
short answer: 4.192679688981604
long answer w/html: 375.7233928331768
long answer: 263.73842437256747
short: 7033 long: 9952 all: 20000
short answer w/html: 4.210009953078345
short answer: 4.206313095407365
long answer w/html: 371.6472065916399
long answer: 260.8560088424437
short: 8830 long: 12471 all: 25000
short answer w/html: 4.242355605889014
short answer: 4.2394110985277464
long answer w/html: 366.95164782294927
long answer: 258.90866811001524
short: 10556 long: 14948 all: 30000
short answer w/html: 4.240147783251231
short answer: 4.237495263357332
long answer w/htm

In [8]:

def average_length(examples):
    short_answer_len = 0
    long_answer_len = 0
    short_answer_count = 0
    long_answer_count = 0
    par_count = 0
    doc_char_length = 0
    doc_word_length = 0
    short_answer_word_len = 0
    long_answer_word_len = 0
    index = 0
    for example in examples:
        index += 1
        par_count += len(example['passage_candidates']['end_positions'])
        doc_char_length += example['passage_candidates']['end_positions'][-1] # - example['passage_candidates']['start_positions'][0])
        doc_word_length += len(example["context"][0].split())
        if example['target']['start_positions'][0] != -1:
            short_answer_count += 1
            short_answer_len += (example['target']['end_positions'][0] - example['target']['start_positions'][0])
            short_answer_word_len += len(example["context"][0][example['target']['start_positions'][0]:example['target']['end_positions'][0]].split())
        if example['target']['passage_indices'][0] != -1:
            long_answer_count += 1
            long_answer_len += (example['passage_candidates']['end_positions'][example['target']['passage_indices'][0]] - example['passage_candidates']['start_positions'][example['target']['passage_indices'][0]])
            long_answer_word_len += len(example["context"][0][example['passage_candidates']['start_positions'][example['target']['passage_indices'][0]]:example['passage_candidates']['end_positions'][example['target']['passage_indices'][0]]].split())
        if index % 10000 == 0:
            print(f'short: {short_answer_count} long: {long_answer_count} all: {index}')
            if short_answer_count > 0:
                print(f'short answer: {short_answer_len/short_answer_count}')
                print(f"short_answer word_count {short_answer_word_len/index}")
            if long_answer_count > 0:
                print(f'long answer: {long_answer_len/long_answer_count}')
                print(f"long_answer word_count {long_answer_word_len/index}")
            print(f"par_count {par_count/index}")
            print(f"doc_char_count {doc_char_length/index}")
            print(f"doc_word_count {doc_word_length/index}")
            print("-----")
    print(f'short: {short_answer_count} long: {long_answer_count} all: {index}')
    if short_answer_count > 0:
        print(f'short answer: {short_answer_len/short_answer_count}')
        print(f"short_answer word_count {short_answer_word_len/index}")
    if long_answer_count > 0:
        print(f'long answer: {long_answer_len/long_answer_count}')
        print(f"long_answer word_count {long_answer_word_len/index}")
    print(f"par_count {par_count/index}")
    print(f"doc_char_count {doc_char_length/index}")
    print(f"doc_word_count {doc_word_length/index}")
    print("-----")

average_length(train_examples)

short: 0 long: 0 all: 10000
par_count 2.2766
doc_char_count 1723.3851
doc_word_count 249.8619
-----
short: 0 long: 0 all: 20000
par_count 2.25515
doc_char_count 1722.58995
doc_word_count 250.99385
-----
short: 4733 long: 6351 all: 30000
short answer: 151.2736108176632
short_answer word_count 3.2511666666666668
long answer: 1531.370335380255
long_answer word_count 37.053666666666665
par_count 2.6476
doc_char_count 2331.9485666666665
doc_word_count 298.6249
-----
short: 12161 long: 16351 all: 40000
short answer: 153.16380231888826
short_answer word_count 6.399125
long answer: 1554.5266956149471
long_answer word_count 72.989175
par_count 3.04345
doc_char_count 2987.8016
doc_word_count 349.6064
-----
short: 17740 long: 23649 all: 47298
short answer: 150.9435738444194
short_answer word_count 7.904943126559263
long answer: 1584.1715928791916
long_answer word_count 93.31400904900842
par_count 3.1340225802359507
doc_char_count 3164.00685018394
doc_word_count 366.60247790604257
-----
